In [2]:
import sys 
sys.path.append('../../')  
from main import device_process_measurement_tomography, Quantities, Cross_Quantities
import QuantumTomography as qt 
import matplotlib.pyplot as plt 
import numpy as np 
from qiskit import QuantumCircuit, Aer, IBMQ 
from qiskit.compiler import transpile 
import json 
from qiskit.result import Result 
from qiskit.test.mock import FakeCasablanca 
from utils_qiskit import * 
from utils_plots import *
from qiskit.providers.ibmq.managed import IBMQJobManager

In [3]:
IBMQ.load_account()
provider    = IBMQ.get_provider(hub='ibm-q-research-2', group='csic-1', project='main')
backend     = provider.get_backend('ibm_perth')
job_manager = IBMQJobManager() 

In [4]:
jobs = [ '00d3107d150c4856bb145bc3fca2f147-16647947252239227',
            '7ce83169d75d4115a8dd2e839692d898-16655968534771705',
            '5373eb205b0e40f180632c1e0774aae4-1666877735257145',
            '04bb65c3638d4ee2bcb2039cf79f24fc-1666877752352261',
            'fce7c304165046509024a820e01ed340-16668777693586967' ]

In [6]:
for job in jobs:

    result_set = job_manager.retrieve_job_set(job, provider).results()
    results    = result_set.combine_results()
    save_results( results, job )
    dpmt = device_process_measurement_tomography( backend )
    circuits_dpmt = dpmt.circuits()
    results_tomography = dpmt.fit( results, paralell=True, gate_set=True )         
    np.save( 'data_tomo_'+job, np.array(results_tomography, dtype=object) ) 